# Statsmodels

Statsmodels is the most general go-to library in Python for doing fancy statistical modeling: linear regressions, GLMs, hierarchical models, mixed effects models, etc.; basically, if it's a commonly used tool and it's not in Scipy's `stats` module, it's probably in Statsmodels.  Statsmodels tends to focus on the mode widely-used, standard models; it usually doesn't have cutting-edge or exotic models, but in my experiences, it's still not as mature or well-polished as the other libraries we've covered (scikit-learn, Pandas, etc).

Install statsmodels with:

```bash
conda install statsmodels
```

You can find the documentation at [statsmodels.org](https://www.statsmodels.org/stable/index.html).

## A note about Statsmodels versus R

The general rule of thumb: R has everything Statsmodels has, and more.  But Statsmodels has it all in one place, maintained by one team, with one (generally) consistent API.  R has its models scattered across a bunch of different packages (sometimes competing ones that do the same thing), some of which may be abandoned, some of which use their own idiosyncratic conventions, and so on.  Statsmodels won't have a lot of cutting-edge models and tests, but R will.

If you have a lot of experience using other statistical packages (R, SPSS, Stata, etc.), you might be tempted to build the same model using them, and then using Statsmodels.  This is a great way to learn Statsmodels!  However: you should avoid trying to _validate_ Statsmodels' implementations this way.  There are a lot of pitfalls that lead you astray: different default settings, different mathematical formulations, different choices for low-level math libraries, different thresholds for numeric tolerance, etc.  You can learn a lot by trying to make something from Statsmodels match something from a different implementation, but in order to truly validate the models, you would need to do a pretty thorough audit of the source code.  (Which you could, in theory do--it's all open source!).  Remember: "same" does not mean "correct!"

# Statsmodels vs Scipy and scikit-learn

Scipy's `stats` module is usually what I reach for instead of Statsmodels.  The API tends to be a lot simpler, the tests tend to be a lot faster, and it usually has everything that I personally need.  (With the caveat that I don't do very fancy statistics in my projects).  Scipy's tests are also easier to manipulate programmtically.  But, Statsmodels will give you more detailed output, and has a lot of stuff that Scipy doesn't.

If scikit-learn has an implementation of something that's also in Statsmodels, scikit-learn will usually be faster--sometimes by a _huge_ margin, especially for large datasets.  But, scikit-learn is designed for _predictive modeling,_ not _explanatory modeling_, so it doesn't calculate p-values, confidence intervals, etc (they're not nearly as important as speed, or other evaluation metrics, when doing predictive modeling).  

I generally find Statsmodels lacking in a few areas.  First, its documentation is nowhere near as good as most other big libaries we've looked at.  It's not terrible, but it doesn't give you as much detail on _how to actually use_ things in the library.  Second, I find Statsmodels is much more prone to undocumented behavior, bugs, and strange inconsistencies once you need to do anything too complex with it.  E.g.: I've never managed to get the missing value imputation to work.  It always throws errors, and I can't figure out from the documentation what I'm supposed to do to actually impute missing values.

# Statsmodels Quickstart

Say it with me now: Statsmodels integrates with Pandas `DataFrame`s and Numpy `array`s.  Here's a few examples of basic models, using a synthetically created regression dataset from scikit-learn.

In [1]:
import pandas as pd
from sklearn.datasets import make_regression
X, Y = make_regression(
    n_samples=50,
    n_features=10,
    n_informative=2,
    random_state=0, # so you can get the same output as I do
)
df = pd.DataFrame(X, columns=[f"Col{i}" for i in range(X.shape[1])])
df["Target"] = Y
print(df)

        Col0      Col1      Col2      Col3      Col4      Col5      Col6  \
0   0.355482 -0.807648  0.766663 -1.115897  0.814520 -0.598654 -1.768538   
1  -0.977278  0.410599  0.978738  0.400157  0.950088  1.764052  1.867558   
2  -0.149635  0.407462  0.298238 -1.099401 -0.435154  0.376426 -0.694568   
3   0.684501  1.719589 -0.309114  0.800298  0.370825 -1.446535  1.732721   
4  -0.663478 -0.437820 -0.744755  1.713343  1.126636 -0.068242 -0.098453   
5   1.895889  1.054452  0.465662  0.900826  1.178780 -1.165150  1.488252   
6  -0.390953  2.064493  1.955912 -2.772593  0.493742  0.399046 -0.652409   
7   1.943621  1.480515 -1.270485 -1.347759 -0.413619  1.883151 -1.173123   
8   0.781198  0.676908 -0.542861 -0.493320  1.494485 -1.424061 -1.156182   
9  -0.460720 -0.159573 -0.118164  1.658131 -1.334258 -1.306527  0.666383   
10  0.063262 -0.237922 -0.463596 -0.345982  0.156507 -0.955945 -1.540797   
11 -1.602058  1.543015 -0.643618 -1.374951 -1.104383 -0.353994  0.625231   
12 -0.521189

The `api` module contains all the models you'd want to fit.  It's conventionally imported as follows.

In [2]:
import statsmodels.api as sm

Let's fit a basic model: an ordinary least squares regression to the Numpy `array` data.

In [3]:
lm = sm.OLS(endog=Y, exog=X)
lm = lm.fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          1.271e+30
Date:                Tue, 16 Aug 2022   Prob (F-statistic):                        0.00
Time:                        18:16:07   Log-Likelihood:                          1394.5
No. Observations:                  50   AIC:                                     -2769.
Df Residuals:                      40   BIC:                                     -2750.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Now let's re-fit it using the `DataFrame` data:

In [4]:
lm = sm.OLS(endog=df["Target"], exog=df.drop(columns=["Target"]))
lm = lm.fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:                 Target   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          1.270e+30
Date:                Tue, 16 Aug 2022   Prob (F-statistic):                        0.00
Time:                        18:18:23   Log-Likelihood:                          1394.4
No. Observations:                  50   AIC:                                     -2769.
Df Residuals:                      40   BIC:                                     -2750.
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Note how the only difference between the two approaches is that with a Numpy `array`, our variable names are all in the form "x1", "x2", etc; with a `DataFrame`, the column names get used as variable names in the summary table.

Pretty simple!  Now let's try a more complex model.  Let's fit a GLM with a Bonimial family and the default link function, just to show how it's done.  (let's ignore how this is absolutely not the right model for this data)

In [5]:
glm = sm.GLM(endog=Y, exog=X, family=sm.families.Binomial(link=None))
glm = glm.fit()
print(glm.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   50
Model:                            GLM   Df Residuals:                       40
Model Family:                Binomial   Df Model:                            9
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    inf
Date:                Tue, 16 Aug 2022   Deviance:                   3.6311e+05
Time:                        18:25:09   Pearson chi2:                 2.48e+21
No. Iterations:                     2   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1         -2.034e+12   1.05e+07  -1.94e+05      0.0

C:\Users\andersonh\Miniconda3\envs\PFDA_Month_7\lib\site-packages\statsmodels\genmod\families\links.py:187: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
C:\Users\andersonh\Miniconda3\envs\PFDA_Month_7\lib\site-packages\statsmodels\genmod\families\family.py:1014: RuntimeWarning: invalid value encountered in log
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +
C:\Users\andersonh\Miniconda3\envs\PFDA_Month_7\lib\site-packages\statsmodels\genmod\families\family.py:1014: RuntimeWarning: divide by zero encountered in log
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +


(The results are nonsense, but the point is that the code is pretty simple).

Pretty much everything in Statsmodels follows the above three-line paradigm:

- Create the model, and specify your dependent/independent variables here.  This creates an uninitialized/unfitted model.
- Call `model.fit()`.  This fits the model and returns the fitted instance, which contains the fit statistics.
    - Note: unlike scikit-learn, `.fit()` is NOT an in-place method!  It does not _update_ the model object; it _returns_ a new kind of object that contains the information from when the model was fit.  So you _have_ to use the `model = model.fit()` line--if you just say `model.fit()`, you'll be calculating the results, but not saving them to a variable, so they get immediately discarded.
- Call `model.summary()`, or other methods/attributes on the object, to see the results.
    - You will usually use `.summary()` if you're manually inspecting the results.
    - You'll use other methods to, e.g., pull out just the P-values, or just the coefficients, etc., if you're programmatically manipulating the results.
        - E.g.: run a test for normality, then based on the results, run either a t-test or a Mann-Whitney U-test.  You could do this by hand, but you should probably do this programmatically if you're analyzing anything other than a fixed, never-going-to-be-updated-again dataset.

Much like scikit-learn, the API for Statsmodels is super consistent across the different models.  Each model really only differs in terms of the arguments it takes when you're creating it.

An important note on Statsmodel's terminology.  Rather than "x and y," or "independent and dependent," or "features and target," Statsmodels uses the terms _endogenous_ to refer to the y/dependent/target variables, and _exogenous_ to refer to the x/dependent/feature variables.  These are abbreviated to _endog_ and _exog_ when used as arguments to models.  A helpful mnemonic (courtesy of the [Statsmodels page on the topic](https://www.statsmodels.org/stable/endog_exog.html)): _exogenous_ has an "x" in it, so it's the "x" variable.

There really isn't a whole lot more to the core of the library than that.  Do your data prep using Pandas, Numpy, scikit-learn, pure Python, or whatever else, then just create your Statsmodels model, fit it, and look at the results.  As always, there is a lot more you can do with this library that we'll be going into.  But, probably moreso than any other library we're looking at, the absolute basics will get you very, very far with Statsmodels.

# Formulas

If you're coming from R, you'll be familiar with formula expressions, like:

```R
price ~ weight + mpg + (make * model)
```

Well, Statsmodels supports formulas like this!  There's one major difference: in R, formulas are a dedicated piece of syntax in the language, which make use of some of R's metaprogramming and delayed evaluation tools.  In Statsmodels, all formulas are _strings,_ since Python does not have a built-in notion of formulas, and Python has none of the metaprogramming/delayed evaluation tools that R does.

There are a few syntactic differences between formulas in Statsmodels and R.  Statsmodels uses the [Patsy](https://patsy.readthedocs.io/en/latest/) language to parse formulas, which is very similar to R (identical, for a lot of cases), but has some differences.  You'll need to read the documentation for more details.

Formulas work best when your exogenous variable is a Pandas `DataFrame`, since you can just use the column names in the formulas.  You'll also need to import a different part of Statsmodels.  The conventional import is:

In [7]:
import statsmodels.formula.api as smf

From there, it should all look pretty straightforward.  The model creation step changes, so it now looks like:

```python
model = smf.modelname(data=my_data, formula="y ~ x1 + x2*x3")
```

Most model name are _all lowercase_ when creating them this way.

In [8]:
X_df = pd.DataFrame(X, columns=[f"Col{i}" for i in range(X.shape[1])])
X_df["Target"] = Y

# smf.ols; compare to sm.OLS, which is in all capital letters
lm = smf.ols(
    data=X_df,
    formula="Target ~ Col0 + Col1 + Col2 + Col3 + Col4 + Col5 + Col6 + Col7 + Col8 + Col9 + 1"
)
lm = lm.fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                 Target   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.983e+30
Date:                Tue, 16 Aug 2022   Prob (F-statistic):               0.00
Time:                        18:29:19   Log-Likelihood:                 1444.7
No. Observations:                  50   AIC:                            -2867.
Df Residuals:                      39   BIC:                            -2846.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.732e-14   1.27e-14      1.363      0.1

Since formulas are just strings, you can manipulate them like strings, and construct them programmatically.  You probably shouldn't be doing this, but there might be times when it's useful.

In [9]:
lm = smf.ols(
    data=X_df,
    # *definitely* unnecessary here, but this migth be useful in more
    # unusual circumtances.
    formula="Target ~ " + " + ".join(f"Col{i}" for i in range(X.shape[1])) + "+ 1"
)
lm = lm.fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                 Target   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.983e+30
Date:                Tue, 16 Aug 2022   Prob (F-statistic):               0.00
Time:                        18:32:55   Log-Likelihood:                 1444.7
No. Observations:                  50   AIC:                            -2867.
Df Residuals:                      39   BIC:                            -2846.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.732e-14   1.27e-14      1.363      0.1

Note that you can also construct models like this using the `statsmodels.api` interface.  In fact, the implementations in `statsmodels.formula.api` just call these methods for you in the background.  So it doesn't really matter which convention you use for formulas, but you should pick one and stick to it as much as possible.

In [ ]:
import statsmodels.api as sm
lm = sm.OLS.from_formula(
    data=X_df,
    formula="Target ~ Col0 + Col1 + Col2 + Col3 + Col4 + Col5 + Col6 + Col7 + Col8 + Col9 + 1"
)
lm = lm.fit()
print(lm.summary())

We can also use formulas to specify on-the-fly data transformations.  Consult the Patsy documentation for full details.

In [11]:
import numpy as np
import statsmodels.api as sm
lm = sm.OLS.from_formula(
    data=X_df,
    formula="Target ~ np.log1p(Col0) + np.exp(Col1) + np.sin(Col2) + np.sqrt(Col3 + Col4 + Col5) + Col6 + Col7 + Col8 + Col9 + 1"
)
lm = lm.fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                 Target   R-squared:                       0.731
Model:                            OLS   Adj. R-squared:                  0.462
Method:                 Least Squares   F-statistic:                     2.715
Date:                Tue, 16 Aug 2022   Prob (F-statistic):             0.0896
Time:                        18:39:02   Log-Likelihood:                -83.451
No. Observations:                  17   AIC:                             184.9
Df Residuals:                       8   BIC:                             192.4
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

C:\Users\andersonh\Miniconda3\envs\PFDA_Month_7\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\andersonh\Miniconda3\envs\PFDA_Month_7\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\andersonh\Miniconda3\envs\PFDA_Month_7\lib\site-packages\scipy\stats\_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
